# 要約 
このJupyter Notebookは、Kaggleの「LMSYS - Chatbot Arena 人間による好み予測チャレンジ」に参加するためのモデル推論を行うためのものです。具体的には、大規模言語モデル（LLM）の応答を比較し、どちらの応答がユーザーに好まれるかを予測するタスクに取り組んでいます。

### 取り組む問題
ノートブックは、与えられたテストデータに対して、異なるLLMの応答（モデルAとモデルB）の勝者を予測する問題に焦点を当てています。この予測は、選択肢のうちどちらのモデルの応答が優れているか、または同等であるのかを示す確率を出力することで行われます。

### 使用する手法とライブラリ
- **ライブラリ**:
  - `bitsandbytes`: メモリ効率の良いパラメータ管理を提供。
  - `peft`: モデルのパラメータを効率的にファインチューニングするためのラッパー。
  - `transformers`: 事前学習済みのトークナイザーやモデルを使用。
  - `pandas` と `numpy`: データ操作や数値計算を行うための一般的なライブラリ。
  - `torch`: モデルの構築や訓練を行うための主要な深層学習ライブラリ。
  - `datasets`: データセットの処理に特化したライブラリ。

- **手法**:
  1. **データのインポートと前処理**: テストデータを読み込み、応答やプロンプトから不要な記号を削除し、モデルの入力形式に整形します。
  2. **トークナイゼーション**: ユーザーのプロンプトとモデルの応答をトークン化し、モデルが処理できる形式に変換します。
  3. **モデルの読み込み**: 事前学習済みのLLMを読み込み、2つのGPUデバイスを使って異なるモデルを並行して評価します。
  4. **推論**: 推論関数を使用して、トークン化されたデータに対してモデルを評価し、モデルAとモデルBの応答の勝者確率を計算します。
  5. **結果の整理**: 推論結果をデータフレームに格納し、不要な列を削除して、最終的な出力ファイル（`submission.csv`）を作成します。

全体として、このノートブックは、効率的なデータ処理と推論を通じて、ユーザーの好みを予測する能力を強化することを目的としています。

---


# 用語概説 
以下は、提供されたJupyter Notebookにおける機械学習や深層学習に関連する専門用語の解説です。初心者がつまずきそうなマイナーな用語や、実務経験がないと馴染みのないものに焦点を当てています。

1. **PEFT (Parameter-Efficient Fine-Tuning)**:
   - モデルの全パラメータを更新せずに、そのサブセットを効率的に微調整する手法。これにより、トレーニング時間やリソースを削減しながら、特定のタスクにモデルを適応させることができます。

2. **BitsAndBytesConfig**:
   - モデルのメモリ効率を向上させるための設定オブジェクト。この設定を使って、モデルを8ビットの精度でロードしたり、計算時のデータ型を指定したりします。

3. **Autocast**:
   - PyTorchにおける自動混合精度の機能。これにより、計算を半精度（16ビット）で行い、必要に応じて全精度（32ビット）に切り替えることで、計算速度とメモリ使用量を最適化します。

4. **トークナイザー (Tokenizer)**:
   - テキストデータを、モデルが理解できる数値形式（トークン）に変換する役割を持つコンポーネント。プロンプトやレスポンスを処理する際に使用されます。

5. **Attention Mask**:
   - 入力シーケンスの各トークンに対して、モデルがどのトークンに注意を払うべきかを示すマスク。通常、パディングトークンを無視するために使用され、1（注視する）または0（無視する）の値を持ちます。

6. **Logits**:
   - モデルが出力する生のスコアまたは値。この値は、最終的に確率に変換される前の未正規化の値です。

7. **Merge and Unload**:
   - 複数のモデルやパラメータを統合し、メモリ上から不要なデータを削除するプロセス。これにより、メモリ使用量を効率的に管理します。

8. **ガーベジコレクション (Garbage Collection)**:
   - プログラム実行中に不要になったメモリを自動的に解放するプロセス。これにより、メモリリークを防ぎ、プログラムの効率を向上させます。

9. **ストレージデータ並列処理 (Storage Data Parallel, SDP)**:
   - 複数のGPUにデータを分散して処理する手法。大きなモデルを効率的にトレーニングするために、データの分散とストレージの共有を行います。

10. **パディングトークン (Padding Token)**:
    - テキスト長を揃えるために追加される特別なトークン。主にバッチ処理の際に、各シーケンスを同じ長さにするために使われます。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq
!pip install -U /kaggle/input/peft-wheel/pytorch/version1/1/peft-0.10.0-py3-none-any.whl -qq
```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なライブラリをインストールするためのコマンドです。
# 'bitsandbytes'というライブラリを指定されたホイールファイルからインストールします。
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

# 次に、'peft'というライブラリを同様に指定されたホイールファイルからインストールします。
!pip install -U /kaggle/input/peft-wheel/pytorch/version1/1/peft-0.10.0-py3-none-any.whl -qq
```

</div>
</details>

In [ ]:
# 必要なライブラリをインストールするためのコマンドです。
# 'bitsandbytes'というライブラリを指定されたホイールファイルからインストールします。
!pip install -U /kaggle/input/bitsandbytes-0-42-0-py3-none-any-whl/bitsandbytes-0.42.0-py3-none-any.whl -qq

# 次に、'peft'というライブラリを同様に指定されたホイールファイルからインストールします。
!pip install -U /kaggle/input/peft-wheel/pytorch/version1/1/peft-0.10.0-py3-none-any.whl -qq

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd
import numpy as np
from scipy.special import softmax
from transformers import AutoTokenizer, AutoModelForSequenceClassification,LlamaForSequenceClassification, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
import torch
from torch.cuda.amp import autocast
from datasets import Dataset
import torch.nn.functional as F
from threading import Thread
import gc
```

</div>
<div class="column-right">

# 日本語訳

```python
# 必要なライブラリをインポートするためのコードです。

# pandasはデータ処理に便利なライブラリです。
import pandas as pd

# numpyは数値計算や配列処理のためのライブラリです。
import numpy as np

# softmax 関数は、入力された値を確率分布に変換します。
from scipy.special import softmax

# transformersライブラリから、事前学習済みのトークナイザーとモデルをインポートします。
from transformers import AutoTokenizer, AutoModelForSequenceClassification, LlamaForSequenceClassification, BitsAndBytesConfig

# peftライブラリからPeftModelとPeftConfigをインポートします。
from peft import PeftModel, PeftConfig

# PyTorchをインポートします。これにより、深層学習モデルを構築・訓練できます。
import torch

# 自動混合精度を使用するためのモジュールをインポートします。
from torch.cuda.amp import autocast

# datasetsライブラリからDatasetクラスをインポートします。これを使ってデータセットを扱います。
from datasets import Dataset

# PyTorchの機能を活用するための関数をインポートします。
import torch.nn.functional as F

# スレッドを作成するためのモジュールです。
from threading import Thread

# メモリ管理のためにガーベジコレクションを行うためのモジュールです。
import gc
```

</div>
</details>

In [ ]:
# 必要なライブラリをインポートするためのコードです。

# pandasはデータ処理に便利なライブラリです。
import pandas as pd

# numpyは数値計算や配列処理のためのライブラリです。
import numpy as np

# softmax 関数は、入力された値を確率分布に変換します。
from scipy.special import softmax

# transformersライブラリから、事前学習済みのトークナイザーとモデルをインポートします。
from transformers import AutoTokenizer, AutoModelForSequenceClassification, LlamaForSequenceClassification, BitsAndBytesConfig

# peftライブラリからPeftModelとPeftConfigをインポートします。
from peft import PeftModel, PeftConfig

# PyTorchをインポートします。これにより、深層学習モデルを構築・訓練できます。
import torch

# 自動混合精度を使用するためのモジュールをインポートします。
from torch.cuda.amp import autocast

# datasetsライブラリからDatasetクラスをインポートします。これを使ってデータセットを扱います。
from datasets import Dataset

# PyTorchの機能を活用するための関数をインポートします。
import torch.nn.functional as F

# スレッドを作成するためのモジュールです。
from threading import Thread

# メモリ管理のためにガーベジコレクションを行うためのモジュールです。
import gc

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
torch.backends.cuda.enable_mem_efficient_sdp(True)
torch.backends.cuda.enable_flash_sdp(True)

MODEL_NAME = "/kaggle/input/llama-3/transformers/8b-hf/1"
MAX_LENGTH = 1284
BATCH_SIZE = 4
```

</div>
<div class="column-right">

# 日本語訳

```python
# GPUのメモリ効率を改善するための設定です。
# ストレージ記憶に適したデータ並列処理を有効にします。
torch.backends.cuda.enable_mem_efficient_sdp(True)

# フラッシュSDP（ストレージデータ並列処理）を有効にします。
torch.backends.cuda.enable_flash_sdp(True)

# モデルが保存されているディレクトリのパスを指定します。
MODEL_NAME = "/kaggle/input/llama-3/transformers/8b-hf/1"

# モデルの入力シーケンスの最大長を指定します。この長さを超えるテキストは切り捨てられます。
MAX_LENGTH = 1284

# バッチサイズを指定します。バッチサイズは、モデルに一度に渡すデータの量を決定します。
BATCH_SIZE = 4
```

</div>
</details>

In [ ]:
# GPUのメモリ効率を改善するための設定です。
# ストレージ記憶に適したデータ並列処理を有効にします。
torch.backends.cuda.enable_mem_efficient_sdp(True)

# フラッシュSDP（ストレージデータ並列処理）を有効にします。
torch.backends.cuda.enable_flash_sdp(True)

# モデルが保存されているディレクトリのパスを指定します。
MODEL_NAME = "/kaggle/input/llama-3/transformers/8b-hf/1"

# モデルの入力シーケンスの最大長を指定します。この長さを超えるテキストは切り捨てられます。
MAX_LENGTH = 1284

# バッチサイズを指定します。バッチサイズは、モデルに一度に渡すデータの量を決定します。
BATCH_SIZE = 4

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")
df.head()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 指定されたCSVファイルを読み込みます。
# test.csvファイルは、テストデータを含んでいます。
df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")

# 読み込んだデータフレームの最初の5行を表示します。
# これにより、データの構造や内容を確認できます。
df.head()
```

</div>
</details>

In [ ]:
# 指定されたCSVファイルを読み込みます。
# test.csvファイルは、テストデータを含んでいます。
df = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv")

# 読み込んだデータフレームの最初の5行を表示します。
# これにより、データの構造や内容を確認できます。
df.head()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def transform(row):
    return row.strip('[]')
```

</div>
<div class="column-right">

# 日本語訳

```python
# 行データを変換するための関数を定義します。
def transform(row):
    # 文字列の前後から角括弧（[]）を削除して返します。
    return row.strip('[]')  # 余分な括弧を取り除くことで、データをクリーンにします。
```

</div>
</details>

In [ ]:
# 行データを変換するための関数を定義します。
def transform(row):
    # 文字列の前後から角括弧（[]）を削除して返します。
    return row.strip('[]')  # 余分な括弧を取り除くことで、データをクリーンにします。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df['prompt'] = df['prompt'].apply(transform)
df['response_a'] = df['response_a'].apply(transform)
df['response_b'] = df['response_b'].apply(transform)
```

</div>
<div class="column-right">

# 日本語訳

```python
# データフレームの特定の列に対して、先ほど定義したtransform関数を適用します。
# 'prompt'列の各要素にtransform関数を適用して角括弧を取り除きます。
df['prompt'] = df['prompt'].apply(transform)

# 'response_a'列の各要素にtransform関数を適用して角括弧を取り除きます。
df['response_a'] = df['response_a'].apply(transform)

# 'response_b'列の各要素にtransform関数を適用して角括弧を取り除きます。
df['response_b'] = df['response_b'].apply(transform)
```

</div>
</details>

In [ ]:
# データフレームの特定の列に対して、先ほど定義したtransform関数を適用します。
# 'prompt'列の各要素にtransform関数を適用して角括弧を取り除きます。
df['prompt'] = df['prompt'].apply(transform)

# 'response_a'列の各要素にtransform関数を適用して角括弧を取り除きます。
df['response_a'] = df['response_a'].apply(transform)

# 'response_b'列の各要素にtransform関数を適用して角括弧を取り除きます。
df['response_b'] = df['response_b'].apply(transform)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df['text'] = 'User prompt: ' + df['prompt'] +  '\n\nModel A :\n' + df['response_a'] +'\n\n----------\n\nModel B:\n'  + df['response_b']
```

</div>
<div class="column-right">

# 日本語訳

```python
# 新しい列'text'をデータフレームに作成します。
# この列にはユーザープロンプトと2つのモデルの応答がフォーマットされた状態で格納されます。
df['text'] = 'User prompt: ' + df['prompt'] +  '\n\nModel A :\n' + df['response_a'] +'\n\n----------\n\nModel B:\n'  + df['response_b'] 

# これにより、各行がユーザーの入力とそれに対するモデルAおよびモデルBの応答を持つ形式になります。
```

</div>
</details>

In [ ]:
# 新しい列'text'をデータフレームに作成します。
# この列にはユーザープロンプトと2つのモデルの応答がフォーマットされた状態で格納されます。
df['text'] = 'User prompt: ' + df['prompt'] +  '\n\nModel A :\n' + df['response_a'] +'\n\n----------\n\nModel B:\n'  + df['response_b'] 

# これにより、各行がユーザーの入力とそれに対するモデルAおよびモデルBの応答を持つ形式になります。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# peft_model_id = "/kaggle/input/lmsys-llama-lora/pytorch/version1/1"
# peft_config = PeftConfig.from_pretrained(peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.pad_token = tokenizer.eos_token
```

</div>
<div class="column-right">

# 日本語訳

```python
# PEFTモデルのIDを指定し、PEFT設定を取得するためのコードです。
# ただしこの行はコメントアウトされています。
# peft_model_id = "/kaggle/input/lmsys-llama-lora/pytorch/version1/1"
# peft_config = PeftConfig.from_pretrained(peft_model_id)

# 事前学習済みのトークナイザーを指定したモデル名から読み込みます。
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# パディングトークンのIDとトークンを設定します。
# パディングトークンは、シーケンスの長さを揃えるために使用されます。
tokenizer.pad_token_id = tokenizer.eos_token_id  # EOS（文の終了）トークンをパディング用に指定
tokenizer.pad_token = tokenizer.eos_token  # EOSトークンを使用してパディングトークンを設定
```

</div>
</details>

In [ ]:
# PEFTモデルのIDを指定し、PEFT設定を取得するためのコードです。
# ただしこの行はコメントアウトされています。
# peft_model_id = "/kaggle/input/lmsys-llama-lora/pytorch/version1/1"
# peft_config = PeftConfig.from_pretrained(peft_model_id)

# 事前学習済みのトークナイザーを指定したモデル名から読み込みます。
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# パディングトークンのIDとトークンを設定します。
# パディングトークンは、シーケンスの長さを揃えるために使用されます。
tokenizer.pad_token_id = tokenizer.eos_token_id  # EOS（文の終了）トークンをパディング用に指定
tokenizer.pad_token = tokenizer.eos_token  # EOSトークンを使用してパディングトークンを設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def tokenize_function(df):
    result = tokenizer(df, padding="max_length", truncation=True, max_length = MAX_LENGTH)
    return result['input_ids'], result['attention_mask']
```

</div>
<div class="column-right">

# 日本語訳

```python
# データフレームのテキストをトークナイズ（分割）するための関数を定義します。
def tokenize_function(df):
    # トークナイザーを使用して、テキストをトークン化します。
    # paddingは最大長に合わせて行い、長さが超えた場合は切り詰めます。
    result = tokenizer(df, padding="max_length", truncation=True, max_length=MAX_LENGTH)

    # トークナイズした結果からinput_ids（トークンID）とattention_mask（注目マスク）を返します。
    return result['input_ids'], result['attention_mask']  # input_ids: 入力トークンのID, attention_mask: パディング位置を示すマスク
```

</div>
</details>

In [ ]:
# データフレームのテキストをトークナイズ（分割）するための関数を定義します。
def tokenize_function(df):
    # トークナイザーを使用して、テキストをトークン化します。
    # paddingは最大長に合わせて行い、長さが超えた場合は切り詰めます。
    result = tokenizer(df, padding="max_length", truncation=True, max_length=MAX_LENGTH)

    # トークナイズした結果からinput_ids（トークンID）とattention_mask（注目マスク）を返します。
    return result['input_ids'], result['attention_mask']  # input_ids: 入力トークンのID, attention_mask: パディング位置を示すマスク

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
temp = df['text'].apply(tokenize_function)
df['input_ids'] = temp.apply(lambda x: x[0])
df['attention_mask'] = temp.apply(lambda x: x[0])
```

</div>
<div class="column-right">

# 日本語訳

```python
# データフレームの'text'列に対してトークナイズ関数を適用します。
temp = df['text'].apply(tokenize_function)

# トークナイズの結果からinput_ids（トークンID）をデータフレームの新しい列'input_ids'に格納します。
df['input_ids'] = temp.apply(lambda x: x[0])

# トークナイズの結果からattention_mask（注目マスク）をデータフレームの新しい列'attention_mask'に格納します。
# （注意: ここではinput_idsが二重に使用されているため、正しくはx[1]であるべきです。）
df['attention_mask'] = temp.apply(lambda x: x[0])  # これはx[1]に変更する必要があります。
```

</div>
</details>

In [ ]:
# データフレームの'text'列に対してトークナイズ関数を適用します。
temp = df['text'].apply(tokenize_function)

# トークナイズの結果からinput_ids（トークンID）をデータフレームの新しい列'input_ids'に格納します。
df['input_ids'] = temp.apply(lambda x: x[0])

# トークナイズの結果からattention_mask（注目マスク）をデータフレームの新しい列'attention_mask'に格納します。
# （注意: ここではinput_idsが二重に使用されているため、正しくはx[1]であるべきです。）
df['attention_mask'] = temp.apply(lambda x: x[0])  # これはx[1]に変更する必要があります。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
df
```

</div>
<div class="column-right">

# 日本語訳

```python
# データフレームの全体を表示します。
# データの内容や新しく追加した列'example', 'input_ids', 'attention_mask'などを確認できます。
df
```

</div>
</details>

In [ ]:
# データフレームの全体を表示します。
# データの内容や新しく追加した列'example', 'input_ids', 'attention_mask'などを確認できます。
df

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# data = Dataset.from_pandas(df[['text']])
# data = data.map(tokenize_function, batched=True)
# data.set_format(type='torch', columns=['input_ids', 'attention_mask'])
```

</div>
<div class="column-right">

# 日本語訳

```python
# データフレームから特定の列を使ってデータセットを作成するコードです。
# ただし、現在この行はコメントアウトされています。
# data = Dataset.from_pandas(df[['text']])

# 作成したデータセットに対して、トークナイズ関数をバッチ処理で適用します。
# data = data.map(tokenize_function, batched=True)

# データセットの形式を設定します。PyTorch形式で、特定の列を指定します。
# data.set_format(type='torch', columns=['input_ids', 'attention_mask'])
```

</div>
</details>

In [ ]:
# データフレームから特定の列を使ってデータセットを作成するコードです。
# ただし、現在この行はコメントアウトされています。
# data = Dataset.from_pandas(df[['text']])

# 作成したデータセットに対して、トークナイズ関数をバッチ処理で適用します。
# data = data.map(tokenize_function, batched=True)

# データセットの形式を設定します。PyTorch形式で、特定の列を指定します。
# data.set_format(type='torch', columns=['input_ids', 'attention_mask'])

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# data
```

</div>
<div class="column-right">

# 日本語訳

```python
# データセットの内容を表示します。
# 現在のデータセットの構造や含まれているデータの確認を行います。
# ただし、現在この行はコメントアウトされています。
# data
```

</div>
</details>

In [ ]:
# データセットの内容を表示します。
# 現在のデータセットの構造や含まれているデータの確認を行います。
# ただし、現在この行はコメントアウトされています。
# data

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
device0 = torch.device('cuda:0')
device1 = torch.device('cuda:1')

bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 使用するデバイスを指定します。ここでは、GPUデバイスを設定しています。
device0 = torch.device('cuda:0')  # 最初のGPUデバイスを指定
device1 = torch.device('cuda:1')  # 2つ目のGPUデバイスを指定

# BitsAndBytesConfigを使用して、メモリ効率の良いモデルを構成します。
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットでのモデル読み込みを有効にします。
    bnb_8bit_compute_dtype=torch.float16,  # 計算に使用するデータ型を16ビット浮動小数点に設定します。
    bnb_8bit_use_double_quant=False  # ダブル量子化を無効にします。
)
```

</div>
</details>

In [ ]:
# 使用するデバイスを指定します。ここでは、GPUデバイスを設定しています。
device0 = torch.device('cuda:0')  # 最初のGPUデバイスを指定
device1 = torch.device('cuda:1')  # 2つ目のGPUデバイスを指定

# BitsAndBytesConfigを使用して、メモリ効率の良いモデルを構成します。
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットでのモデル読み込みを有効にします。
    bnb_8bit_compute_dtype=torch.float16,  # 計算に使用するデータ型を16ビット浮動小数点に設定します。
    bnb_8bit_use_double_quant=False  # ダブル量子化を無効にします。
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
peft_model_id = "/kaggle/input/lmsys-llama-lora/pytorch/version1/1"
```

</div>
<div class="column-right">

# 日本語訳

```python
# PEFTモデルのIDを指定します。
# このパスは、指定されたディレクトリに格納されているPEFTモデルを参照します。
peft_model_id = "/kaggle/input/lmsys-llama-lora/pytorch/version1/1"
```

</div>
</details>

In [ ]:
# PEFTモデルのIDを指定します。
# このパスは、指定されたディレクトリに格納されているPEFTモデルを参照します。
peft_model_id = "/kaggle/input/lmsys-llama-lora/pytorch/version1/1"

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:0')
model_0.config.pad_token_id = tokenizer.pad_token_id

model_0 = PeftModel.from_pretrained(model_0, peft_model_id).to(device0)
model_0 = model_0.merge_and_unload()
model_0.eval()
```

</div>
<div class="column-right">

# 日本語訳

```python
# Llamaモデルを事前学習済みの重みから読み込みます。
# num_labelsは分類するラベルの数を指定します。
model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 分類ラベルは3つであることを示します。
    torch_dtype=torch.float16,  # モデルのデータ型を16ビット浮動小数点に設定します。
    quantization_config=bnb_config,  # 先ほど設定した量子化の設定を適用します。
    device_map='cuda:0'  # 最初のGPUデバイスに割り当てます。
)

# トークナイザーのパディングトークンIDをモデルの設定に指定します。
model_0.config.pad_token_id = tokenizer.pad_token_id

# PEFTモデルを事前学習済みのPEFT重みから読み込み、指定したデバイスに移行します。
model_0 = PeftModel.from_pretrained(model_0, peft_model_id).to(device0)

# モデルのマージとメモリからのアンロードを行います。
model_0 = model_0.merge_and_unload()

# モデルを評価モードに設定します。これにより、ドロップアウトなどのレイヤーが無効になります。
model_0.eval()
```

</div>
</details>

In [ ]:
# Llamaモデルを事前学習済みの重みから読み込みます。
# num_labelsは分類するラベルの数を指定します。
model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 分類ラベルは3つであることを示します。
    torch_dtype=torch.float16,  # モデルのデータ型を16ビット浮動小数点に設定します。
    quantization_config=bnb_config,  # 先ほど設定した量子化の設定を適用します。
    device_map='cuda:0'  # 最初のGPUデバイスに割り当てます。
)

# トークナイザーのパディングトークンIDをモデルの設定に指定します。
model_0.config.pad_token_id = tokenizer.pad_token_id

# PEFTモデルを事前学習済みのPEFT重みから読み込み、指定したデバイスに移行します。
model_0 = PeftModel.from_pretrained(model_0, peft_model_id).to(device0)

# モデルのマージとメモリからのアンロードを行います。
model_0 = model_0.merge_and_unload()

# モデルを評価モードに設定します。これにより、ドロップアウトなどのレイヤーが無効になります。
model_0.eval()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')
model_1.config.pad_token_id = tokenizer.pad_token_id

model_1 = PeftModel.from_pretrained(model_1, peft_model_id).to(device1)
model_1 = model_1.merge_and_unload()
model_1.eval()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 2つ目のLlamaモデルを事前学習済みの重みから読み込みます。
# num_labelsは分類するラベルの数を指定します。
model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 分類ラベルは3つであることを示します。
    torch_dtype=torch.float16,  # モデルのデータ型を16ビット浮動小数点に設定します。
    quantization_config=bnb_config,  # 先ほど設定した量子化の設定を適用します。
    device_map='cuda:1'  # 2つ目のGPUデバイスに割り当てます。
)

# トークナイザーのパディングトークンIDをモデルの設定に指定します。
model_1.config.pad_token_id = tokenizer.pad_token_id

# PEFTモデルを事前学習済みのPEFT重みから読み込み、指定したデバイスに移行します。
model_1 = PeftModel.from_pretrained(model_1, peft_model_id).to(device1)

# モデルのマージとメモリからのアンロードを行います。
model_1 = model_1.merge_and_unload()

# モデルを評価モードに設定します。これにより、ドロップアウトなどのレイヤーが無効になります。
model_1.eval()
```

</div>
</details>

In [ ]:
# 2つ目のLlamaモデルを事前学習済みの重みから読み込みます。
# num_labelsは分類するラベルの数を指定します。
model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # 分類ラベルは3つであることを示します。
    torch_dtype=torch.float16,  # モデルのデータ型を16ビット浮動小数点に設定します。
    quantization_config=bnb_config,  # 先ほど設定した量子化の設定を適用します。
    device_map='cuda:1'  # 2つ目のGPUデバイスに割り当てます。
)

# トークナイザーのパディングトークンIDをモデルの設定に指定します。
model_1.config.pad_token_id = tokenizer.pad_token_id

# PEFTモデルを事前学習済みのPEFT重みから読み込み、指定したデバイスに移行します。
model_1 = PeftModel.from_pretrained(model_1, peft_model_id).to(device1)

# モデルのマージとメモリからのアンロードを行います。
model_1 = model_1.merge_and_unload()

# モデルを評価モードに設定します。これにより、ドロップアウトなどのレイヤーが無効になります。
model_1.eval()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
gc.collect()
```

</div>
<div class="column-right">

# 日本語訳

```python
# ガーベジコレクションを実行します。
# 不要なメモリを解放し、効率的なメモリ管理を促進するために使用します。
gc.collect()
```

</div>
</details>

In [ ]:
# ガーベジコレクションを実行します。
# 不要なメモリを解放し、効率的なメモリ管理を促進するために使用します。
gc.collect()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def inference(df,model,device,batch_size=BATCH_SIZE):
    
    all_probabilities = []
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        
        batch_input_ids = torch.tensor(df['input_ids'][start_idx:end_idx].tolist()).to(device)
        batch_attention_mask = torch.tensor(df['attention_mask'][start_idx:end_idx].tolist()).to(device)
        
        with torch.no_grad():
            with autocast():
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=-1)
        all_probabilities.extend(probabilities.cpu().numpy())
    
    del batch_input_ids, batch_attention_mask, outputs
    gc.collect()
    torch.cuda.empty_cache()  

    all_probabilities = np.array(all_probabilities)
    
    df['winner_model_a'] = all_probabilities[:, 0]
    df['winner_model_b'] = all_probabilities[:, 1]
    df['winner_tie'] = all_probabilities[:, 2]
    return df
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルを用いた推論を行うための関数を定義します。
def inference(df, model, device, batch_size=BATCH_SIZE):
    
    # 出力結果を格納するリストを初期化します。
    all_probabilities = []
    
    # データフレームをバッチサイズに基づいてループ処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終端インデックスを計算
        
        # バッチごとのinput_idsとattention_maskを取得し、指定したデバイスに転送します。
        batch_input_ids = torch.tensor(df['input_ids'][start_idx:end_idx].tolist()).to(device)
        batch_attention_mask = torch.tensor(df['attention_mask'][start_idx:end_idx].tolist()).to(device)
        
        # 勾配計算を無効にし、混合精度推論を使用します。
        with torch.no_grad():
            with autocast():
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask)
        logits = outputs.logits  # モデルの出力からlogitsを取得
        probabilities = F.softmax(logits, dim=-1)  # logitsにソフトマックスを適用して確率を計算
        all_probabilities.extend(probabilities.cpu().numpy())  # 確率をリストに追加
    
    # バッチのテンソルを削除し、メモリを解放します。
    del batch_input_ids, batch_attention_mask, outputs
    gc.collect()  # ゴミ収集を実行
    torch.cuda.empty_cache()  # CUDAメモリを空にします。

    all_probabilities = np.array(all_probabilities)  # 確率をNumPy配列に変換
    
    # 各モデルの勝者確率をデータフレームに追加します。
    df['winner_model_a'] = all_probabilities[:, 0]  # モデルAの勝者確率
    df['winner_model_b'] = all_probabilities[:, 1]  # モデルBの勝者確率
    df['winner_tie'] = all_probabilities[:, 2]  # 引き分けの確率
    return df  # 更新されたデータフレームを返します。
```

</div>
</details>

In [ ]:
# モデルを用いた推論を行うための関数を定義します。
def inference(df, model, device, batch_size=BATCH_SIZE):
    
    # 出力結果を格納するリストを初期化します。
    all_probabilities = []
    
    # データフレームをバッチサイズに基づいてループ処理します。
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終端インデックスを計算
        
        # バッチごとのinput_idsとattention_maskを取得し、指定したデバイスに転送します。
        batch_input_ids = torch.tensor(df['input_ids'][start_idx:end_idx].tolist()).to(device)
        batch_attention_mask = torch.tensor(df['attention_mask'][start_idx:end_idx].tolist()).to(device)
        
        # 勾配計算を無効にし、混合精度推論を使用します。
        with torch.no_grad():
            with autocast():
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask)
        logits = outputs.logits  # モデルの出力からlogitsを取得
        probabilities = F.softmax(logits, dim=-1)  # logitsにソフトマックスを適用して確率を計算
        all_probabilities.extend(probabilities.cpu().numpy())  # 確率をリストに追加
    
    # バッチのテンソルを削除し、メモリを解放します。
    del batch_input_ids, batch_attention_mask, outputs
    gc.collect()  # ゴミ収集を実行
    torch.cuda.empty_cache()  # CUDAメモリを空にします。

    all_probabilities = np.array(all_probabilities)  # 確率をNumPy配列に変換
    
    # 各モデルの勝者確率をデータフレームに追加します。
    df['winner_model_a'] = all_probabilities[:, 0]  # モデルAの勝者確率
    df['winner_model_b'] = all_probabilities[:, 1]  # モデルBの勝者確率
    df['winner_tie'] = all_probabilities[:, 2]  # 引き分けの確率
    return df  # 更新されたデータフレームを返します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
N_SAMPLES = len(df)

half = round(N_SAMPLES / 2)
sub1 = df.iloc[0:half].copy()
sub2 = df.iloc[half:N_SAMPLES].copy()
```

</div>
<div class="column-right">

# 日本語訳

```python
# データフレームのサンプル数を取得します。
N_SAMPLES = len(df)

# サンプルの数を半分に分割します。丸めて整数値にします。
half = round(N_SAMPLES / 2)

# データフレームを2つの部分に分割します。
sub1 = df.iloc[0:half].copy()  # 最初の半分のデータをコピー
sub2 = df.iloc[half:N_SAMPLES].copy()  # 残りの半分のデータをコピー
```

</div>
</details>

In [ ]:
# データフレームのサンプル数を取得します。
N_SAMPLES = len(df)

# サンプルの数を半分に分割します。丸めて整数値にします。
half = round(N_SAMPLES / 2)

# データフレームを2つの部分に分割します。
sub1 = df.iloc[0:half].copy()  # 最初の半分のデータをコピー
sub2 = df.iloc[half:N_SAMPLES].copy()  # 残りの半分のデータをコピー

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import warnings
warnings.filterwarnings("ignore")
```

</div>
<div class="column-right">

# 日本語訳

```python
# 警告メッセージを無視する設定を行います。
# これにより、警告メッセージが表示されなくなります。
import warnings
warnings.filterwarnings("ignore")
```

</div>
</details>

In [ ]:
# 警告メッセージを無視する設定を行います。
# これにより、警告メッセージが表示されなくなります。
import warnings
warnings.filterwarnings("ignore")

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 推論を実行するための関数を定義します。
def run_inference(df, model, device, results, index):
    # 指定されたインデックスに対して、推論結果を保存します。
    results[index] = inference(df, model, device)  # inference関数を呼び出して結果を取得し、resultsリストに格納
```

</div>
</details>

In [ ]:
# 推論を実行するための関数を定義します。
def run_inference(df, model, device, results, index):
    # 指定されたインデックスに対して、推論結果を保存します。
    results[index] = inference(df, model, device)  # inference関数を呼び出して結果を取得し、resultsリストに格納

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
results = {}
t0 = Thread(target=run_inference, args=(sub1, model_0, device0,results, 0))
t1 = Thread(target=run_inference, args=(sub2, model_1, device1,results, 1))

t0.start()
t1.start()

# Wait for all threads to finish
t0.join()
t1.join()

data = pd.concat([results[0], results[1]], axis=0)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 結果を格納するための辞書を初期化します。
results = {}

# サブデータセットに対して異なるモデルで推論を行うためのスレッドを作成します。
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # モデル0用のスレッド
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # モデル1用のスレッド

# スレッドを起動します。
t0.start()
t1.start()

# すべてのスレッドが終了するのを待ちます。
t0.join()
t1.join()

# 2つの結果を結合して単一のデータフレームにします。
data = pd.concat([results[0], results[1]], axis=0)  # 縦に結合します。
```

</div>
</details>

In [ ]:
# 結果を格納するための辞書を初期化します。
results = {}

# サブデータセットに対して異なるモデルで推論を行うためのスレッドを作成します。
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # モデル0用のスレッド
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # モデル1用のスレッド

# スレッドを起動します。
t0.start()
t1.start()

# すべてのスレッドが終了するのを待ちます。
t0.join()
t1.join()

# 2つの結果を結合して単一のデータフレームにします。
data = pd.concat([results[0], results[1]], axis=0)  # 縦に結合します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
data.drop(columns=['prompt','response_a','response_b','text','input_ids','attention_mask'],axis=1,inplace=True)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 不要な列をデータフレームから削除します。
# 'prompt', 'response_a', 'response_b', 'text', 'input_ids', 'attention_mask'の各列を削除します。
data.drop(columns=['prompt', 'response_a', 'response_b', 'text', 'input_ids', 'attention_mask'], axis=1, inplace=True)  # 'inplace=True' でデータフレームを直接更新します。
```

</div>
</details>

In [ ]:
# 不要な列をデータフレームから削除します。
# 'prompt', 'response_a', 'response_b', 'text', 'input_ids', 'attention_mask'の各列を削除します。
data.drop(columns=['prompt', 'response_a', 'response_b', 'text', 'input_ids', 'attention_mask'], axis=1, inplace=True)  # 'inplace=True' でデータフレームを直接更新します。

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
data
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最終的なデータフレームの内容を表示します。
# 不要な列を削除した後のデータフレームの構造や残っているデータを確認します。
data
```

</div>
</details>

In [ ]:
# 最終的なデータフレームの内容を表示します。
# 不要な列を削除した後のデータフレームの構造や残っているデータを確認します。
data

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
data.to_csv("submission.csv",index=False)
```

</div>
<div class="column-right">

# 日本語訳

```python
# 最終的なデータフレームをCSVファイルとして保存します。
# 'submission.csv'というファイル名で、インデックスは含めずに保存します。
data.to_csv("submission.csv", index=False)
```

</div>
</details>

In [ ]:
# 最終的なデータフレームをCSVファイルとして保存します。
# 'submission.csv'というファイル名で、インデックスは含めずに保存します。
data.to_csv("submission.csv", index=False)